In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import NegotiationTools as negtools
from NegotiationConfigNeuralNet import *
import os
import glob
import imageio
import Experiments as exp
import tensorflow as tf
from itertools import groupby

from multiprocessing import Pool

PRED_PATH = 'predictions/brats2019v2_testing/'
RESULT_PATH = 'results/brats2019v2_testing/'
os.makedirs(RESULT_PATH, exist_ok=True)
os.makedirs(RESULT_PATH+'csv/', exist_ok=True)
GT_LIST = sorted(glob.glob(PRED_PATH+'GT/*')) # Here the files are forted by SAMPLE but not by SLICE

AGENTS = [
 'Segan_IO_TF2_brats2019v2_ALL_ED',
 'Segan_IO_TF2_brats2019v2_ALL_ET',
 'Segan_IO_TF2_brats2019v2_ALL_NCRNET'
]



In [8]:
def run_thread(input_tuple):
    print("Running thread for slice {}: {} {}".format(input_tuple[0], input_tuple[1].shape, input_tuple[2].shape))
    result = exp.run_experiment_on_list(proposals_list=[input_tuple[1]], gt_list=[input_tuple[2]], return_mean=False, agent_names=AGENTS, label_names=['Negative', 'Positive'], return_outputs=True, MAX_STEPS=10)
    result[0]['sample_id'] = input_tuple[0]
    print("thread {} finished".format(input_tuple[0]))
    return 1
    

In [9]:
for sample_id, gt_paths_group in groupby(GT_LIST, key=lambda x: int(os.path.basename(x).replace('.npy', '').split('_')[0])):
    gt_slice_paths = sorted(list(gt_paths_group), key=lambda x: int(os.path.basename(x).replace('.npy', '').split('_')[-1]))

    prediction_slices = list()
    gt_slices = list()
    for gt_sp in gt_slice_paths:
        prediction = list()
        for agent in AGENTS:
            break
    break

In [10]:
pred_binary = np.load(gt_sp.replace('/GT/', '/{}/'.format(agent)))

In [12]:
pred_binary.shape

(160, 160, 1)

In [13]:
gt_binary = np.load(gt_sp)

In [16]:
gt_binary.shape

(160, 160, 1)

In [ ]:
for sample_id, gt_paths_group in groupby(GT_LIST, key=lambda x: int(os.path.basename(x).replace('.npy', '').split('_')[0])):
    gt_slice_paths = sorted(list(gt_paths_group), key=lambda x: int(os.path.basename(x).replace('.npy', '').split('_')[-1]))

    prediction_slices = list()
    gt_slices = list()

    for gt_sp in gt_slice_paths:
        prediction = list()
        for agent in AGENTS:
            pred_binary = np.load(gt_sp.replace('/GT/', '/{}/'.format(agent))) # Binary representation
            pred_softmax = np.concatenate([1-pred_binary, pred_binary], axis=-1) # One_hot (Not tumor, Tumor) representation
            prediction.append(pred_softmax)
        prediction = np.stack(prediction)
        prediction_slices.append(prediction)
        gt_binary = np.load(gt_sp)
        gt_slices.append(np.concatenate([1-gt_binary, gt_binary], axis=-1))
    
    thread_input = list(zip([i for i in range(len(prediction_slices))], prediction_slices, gt_slices))
    pool = Pool(processes=16)
    thread_results = pool.map(run_thread, thread_input)
    
#     results, outputs = exp.run_experiment_on_list(proposals_list=prediction_slices[:5], gt_list=gt_slices[:5], return_mean=False, agent_names=AGENTS, label_names=['Negative', 'Positive'], return_outputs=True, MAX_STEPS=10)
#     results['gt_id'] = sample_id
    
#     results.to_csv(RESULT_PATH+'csv/'+ str(sample_id) + '.csv')
#     for output in outputs:
#         for method_name, outp in output.items():
#             os.makedirs(RESULT_PATH+method_name+'/', exist_ok=True)
#             if isinstance(outp, dict):
#                 for agr_prop in outp.keys():
#                     np.save(RESULT_PATH+method_name+'/' + str(sample_id) + '_' + agr_prop + '.npy', outp[agr_prop])
#             else:
#                 np.save(RESULT_PATH+method_name+'/' + str(sample_id)+'.npy', outp.astype(np.float32))
        
    print("Done sample {}".format(str(sample_id)))
    break

In [15]:
os.cpu_count()

16

In [11]:
thread_input[0][0]

0